In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
import string

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context


# nltk.download('punkt')
# nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [5]:
df = pd.read_csv("reduced_reddit_posts.csv")

df = df[(df["selftext"] != '') & (df["selftext"] != "[removed]") & (df["selftext"] != "[deleted]")].dropna(subset=["selftext"]).reset_index(drop=True)
df = df.drop(["num_crossposts", "upvote_ratio", "num_comments", "url", "score"], axis=1)
df

,id,author,created_utc,title,selftext
0,vdcl0,minerva_K,1340259437,is this thing on?,anybody awake?
1,12bic1,[deleted],1351567414,Turned 40 last July and started have hot flash...,My mother said she went into full blown menopa...
2,13fiwk,[deleted],1353294608,winter shoes for mom,my mom has osteoarthritis and i want to get he...
3,15jjqu,oldmanjoe,1356649991,Sex after Menopause?,"My wife went through this ""wonderful"" change, ..."
4,19wwpb,[deleted],1362754970,How does one live with someone going through M...,My wife (early 50s and has been diagnosed by h...
...,...,...,...,...,...
11597,1004teb,Rosie4268,1672524104,I'm so over this moodiness,I'm in peri and have been for some time. Moodi...
11598,10058dp,pumaboxbug2,1672525367,How can we make 2023 kinder to us than 2022?,What are non-medicated ways to ease the depres...
11599,1005nki,Street-Lab-9570,1672526636,Hot flashes,I'm 54 post meno and taking estrodot 75 mg and...
11600,1006cfo,One-Reflection-6779,1672528757,Perimenopause - change in vaginal odor?,"Hi everyone,\n\nHas anyone else noticed a chan..."


In [6]:
def epoch_to_date(epoch_time):
    """
    Convert epoch time to datetime object with only year, month, and day.

    :param epoch_time: Epoch time in seconds
    :type epoch_time: int or float
    :return: Date string in the format 'YYYY-MM-DD'
    :rtype: str
    """
    dt = datetime.fromtimestamp(epoch_time)
    return dt.strftime('%Y-%m-%d')

df["created_utc"] = df["created_utc"].apply(epoch_to_date)

In [7]:
df['tokenized_title'] = df['title'].apply(lambda x: word_tokenize(x))
df['tokenized_text'] = df['selftext'].apply(lambda x: word_tokenize(x))

stop_words = set(stopwords.words('english'))

df['tokenized_title'] = df['tokenized_title'].apply(lambda x: [word.lower() for word in x if word.lower() not in stop_words])
df['tokenized_title'] = df['tokenized_title'].apply(lambda x: [word for word in x if word not in string.punctuation])
df['tokenized_title'] = df['tokenized_title'].apply(lambda x: [word for word in x if word.isalnum()])

df['tokenized_text'] = df['tokenized_text'].apply(lambda x: [word.lower() for word in x if word.lower() not in stop_words])
df['tokenized_text'] = df['tokenized_text'].apply(lambda x: [word for word in x if word not in string.punctuation])
df['tokenized_text'] = df['tokenized_text'].apply(lambda x: [word for word in x if word.isalnum()])
df

,id,author,created_utc,title,selftext,tokenized_title,tokenized_text
0,vdcl0,minerva_K,2012-06-20,is this thing on?,anybody awake?,[thing],"[anybody, awake]"
1,12bic1,[deleted],2012-10-29,Turned 40 last July and started have hot flash...,My mother said she went into full blown menopa...,"[turned, 40, last, july, started, hot, flashes...","[mother, said, went, full, blown, menopause, a..."
2,13fiwk,[deleted],2012-11-18,winter shoes for mom,my mom has osteoarthritis and i want to get he...,"[winter, shoes, mom]","[mom, osteoarthritis, want, get, boots, shoes,..."
3,15jjqu,oldmanjoe,2012-12-27,Sex after Menopause?,"My wife went through this ""wonderful"" change, ...","[sex, menopause]","[wife, went, wonderful, change, longer, sex, u..."
4,19wwpb,[deleted],2013-03-08,How does one live with someone going through M...,My wife (early 50s and has been diagnosed by h...,"[one, live, someone, going, menopause]","[wife, early, 50s, diagnosed, doctor, going, c..."
...,...,...,...,...,...,...,...
11597,1004teb,Rosie4268,2022-12-31,I'm so over this moodiness,I'm in peri and have been for some time. Moodi...,[moodiness],"[peri, time, moodiness, period, always, thing,..."
11598,10058dp,pumaboxbug2,2022-12-31,How can we make 2023 kinder to us than 2022?,What are non-medicated ways to ease the depres...,"[make, 2023, kinder, us, 2022]","[ways, ease, depression, anxiety, diminishing,..."
11599,1005nki,Street-Lab-9570,2022-12-31,Hot flashes,I'm 54 post meno and taking estrodot 75 mg and...,"[hot, flashes]","[54, post, meno, taking, estrodot, 75, mg, 100..."
11600,1006cfo,One-Reflection-6779,2022-12-31,Perimenopause - change in vaginal odor?,"Hi everyone,\n\nHas anyone else noticed a chan...","[perimenopause, change, vaginal, odor]","[hi, everyone, anyone, else, noticed, change, ..."
